### Importação de bibliotecas

In [1]:
import requests as req
import pandas as pd
import re as reg
import sqlite3 as slite

### conexão com o banco de dados

In [17]:
conn = slite.connect("db_star_wars")

In [3]:
characters = req.get('https://swapi.dev/api/people')
planets = req.get('https://swapi.dev/api/planets')
films = req.get('https://swapi.dev/api/films')

In [4]:
df_charcter = pd.json_normalize(characters.json(), 'results')
df_planets = pd.json_normalize(planets.json(), 'results')
df_films = pd.json_normalize(films.json(), 'results')

In [22]:
df_charcter.count()

name           10
height         10
mass           10
hair_color     10
skin_color     10
eye_color      10
birth_year     10
gender         10
homeworld      10
films          10
species        10
vehicles       10
starships      10
created        10
edited         10
url            10
characterID    10
planetID       10
filmsID        10
dtype: int64

In [5]:
def extractNumber(url):
    found = reg.search(r'/(\d+)/', url)
    return int(found.group(1)) if found else None

In [6]:

df_charcter['characterID'] = df_charcter['url'].apply(extractNumber)
df_planets['planetID'] = df_planets['url'].str.extract(r'/(\d+)/')
df_charcter['planetID'] = df_charcter['homeworld'].str.extract(r'/(\d+)/')

df_films['filmsID'] = df_planets['url'].str.extract(r'/(\d+)/')
df_charcter['filmsID'] = df_charcter['films'].apply(lambda x: [extractNumber(url) for url in x])

In [7]:
df_charcter['gender'] = df_charcter['gender'].replace('n/a', '')
df_charcter.head(2)


,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url,characterID,planetID,filmsID
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/,1,1,"[1, 2, 3, 6]"
1,C-3PO,167,75,n/a,gold,yellow,112BBY,,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.dev/api/people/2/,2,1,"[1, 2, 3, 4, 5, 6]"


In [8]:
df_films.head(2)

,title,episode_id,opening_crawl,director,producer,release_date,characters,planets,starships,vehicles,species,created,edited,url,filmsID
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/planets/1/, https://swa...","[https://swapi.dev/api/starships/2/, https://s...","[https://swapi.dev/api/vehicles/4/, https://sw...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/,1
1,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/planets/4/, https://swa...","[https://swapi.dev/api/starships/3/, https://s...","[https://swapi.dev/api/vehicles/8/, https://sw...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-12T11:26:24.656000Z,2014-12-15T13:07:53.386000Z,https://swapi.dev/api/films/2/,2


In [9]:
df_planets.head(2)

,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url,planetID
0,Tatooine,23,304,10465,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/,1
1,Alderaan,24,364,12500,temperate,1 standard,"grasslands, mountains",40,2000000000,"[https://swapi.dev/api/people/5/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T11:35:48.479000Z,2014-12-20T20:58:18.420000Z,https://swapi.dev/api/planets/2/,2


In [10]:
df_charcter_expanded = df_charcter.explode('filmsID')
df_charcter_film = df_charcter_expanded[['characterID', 'filmsID']]

In [42]:

conn.execute("""
    DROP TABLE IF EXISTS planets
""")
conn.execute("""
    DROP TABLE IF EXISTS films
""")
conn.execute("""
    DROP TABLE IF EXISTS characters
""")
conn.commit()

In [43]:
conn.execute("""
    CREATE TABLE IF NOT EXISTS planets(
        planetID INTEGER NOT NULL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        rotation_period INTEGER NOT NULL,
        orbital_period INTEGER,
        diameter INTEGER,
        climate VARCHAR(15),
        gravity VARCHAR(30),
        terrain VARCHAR(60),
        surface_water INTEGER
    )
""")


conn.execute("""
    CREATE TABLE IF NOT EXISTS films(
        filmID INTEGER NOT NULL PRIMARY KEY,
        title VARCHAR(255),
        release_date DATE
    )
""")

conn.execute("""
    CREATE TABLE IF NOT EXISTS characters(
        characterID INTEGER NOT NULL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        height INTEGER,
        mass INTEGER,
        hair_color VARCHAR(15),
        skin_color VARCHAR(15),
        eye_color VARCHAR(15),
        birth_year VARCHAR(30),
        gender VARCHAR(15)
    )
""")

conn.commit()

In [44]:
data_characters = df_charcter.to_dict(orient='records')
data_films = df_films.to_dict(orient='records')
data_planets = df_planets.to_dict(orient='records')

In [49]:

for films in data_films:
    conn.execute(
        """INSERT INTO films (filmID, title, release_date) VALUES (?, ?, ?) """,
            (
                films['filmsID'],
                films['title'],
                films['release_date'],
            )
    )
    conn.commit()

for planet in data_planets:
    conn.execute(
        """INSERT INTO planets (planetID, name, rotation_period, orbital_period, diameter, climate, gravity, terrain, surface_water) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?) """,
            (
                planet['planetID'],
                planet['name'],
                planet['rotation_period'],
                planet['orbital_period'],
                planet['diameter'],
                planet['climate'],
                planet['gravity'],
                planet['terrain'],
                planet['surface_water']
            )
    )
    conn.commit()




for character in data_characters:
    conn.execute(
        """INSERT INTO characters (characterID, name, height, mass, hair_color, skin_color, eye_color, birth_year, gender) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?) """,
            (
                character['characterID'],
                character['name'],
                character['height'],
                character['mass'],
                character['hair_color'],
                character['skin_color'],
                character['eye_color'],
                character['birth_year'],
                character['gender']
            )
    )
    conn.commit()